## 모델 성능 검증
* 1986년 제프리 힌튼 교수가 오차 역전파를 발표한 직후, 
* 존스 홉킨스의 세즈노프스키(Sejnowski) 교수는 오차 역전파가 은닉층의 가중치를 
  실제로 업데이트시키는 것을 확인하고 싶었습니다. 
* 그는 광석과 일반 암석에 수중 음파 탐지기를 쏜 후 결과를 모아 데이터셋을 준비했고, 
* 음파 탐지기의 수신 결과만 보고 광석인지 일반 암석인지를 구분하는 모델을 만들
* 세즈노프스키 교수가 했던 초음파 광물 예측 실험을 텐서플로로 재현해 보고, 
* 이렇게 구해진 실험 정확도를 평가하는 방법과 성능을 향상시키는 
  중요한 머신 러닝 기법들에 대해 알아

## 데이터 적재

In [1]:
# pandas 라이브러리를 불러옵니다.
import pandas as pd

# 광물 데이터를 불러옵니다.
df = pd.read_csv('data/sonar3.csv', header=None)

df.head() # 첫 다섯 줄을 봅니다.

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


In [2]:
# 일반 암석과 광석이 각각 몇 개나 포함되어 있는지 알아
df[60].value_counts()

1    111
0     97
Name: 60, dtype: int64

In [3]:
# 1~60번째 열을 X 변수에 저장하고 광물의 종류는 y로 저장
X = df.iloc[:,0:60]
y = df.iloc[:,60]

## 딥러닝 모델 생성 및 예측

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 모델을 설정합니다.
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

C:\Java\miniconda3\envs\.py39\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [5]:
# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', 
metrics=['accuracy'])

In [6]:
# 모델을 실행합니다.
history = model.fit(X, y, epochs=200, batch_size=10, verbose=False)

In [25]:
hist_df = pd.DataFrame(history.history)
hist_df.head()

,loss,accuracy
0,0.701026,0.485030
1,0.680813,0.574850
2,0.669224,0.550898
3,0.657328,0.610778
4,0.645808,0.604790


In [7]:
# 정확도 확인
model.evaluate(X, y)[1]

7/7 [==============================] - 0s 2ms/step - loss: 0.0321 - accuracy: 0.9952


0.995192289352417

## 과적합 이해하기
* 모델이 학습 데이터셋 안에서는 일정 수준 이상의 예측 정확도를 보이지만, 
* 새로운 데이터에 적용하면 잘 맞지 않는 것을 의미
* 과적합은 층이 너무 많거나 변수가 복잡해서 발생하기도 하고 
* 테스트셋과 학습셋이 중복될 때 생기기도 합니다. 
* 특히 딥러닝은 학습 단계에서 입력층, 은닉층, 출력층의 노드들에 상당히 많은 변수가 투입
* 따라서 딥러닝을 진행하는 동안 과적합에 빠지지 않게 늘 주의해야 합니다.

## 학습셋과 테스트셋
* 과적합을 방지하려면 먼저 학습을 하는 데이터셋과 
* 이를 테스트할 데이터셋을 완전히 구분한 후 
* 학습과 동시에 테스트를 병행하며 진행하는 것이 한 방법
* 예를 들어 데이터셋이 총 100개의 샘플로 이루어져 있다면
    + 신경망을 만들어 70개의 샘플로 학습을 진행한 후 이 학습의 결과를 저장
    + 나머지 30개의 샘플로 실험해서 정확도를 살펴보면 
    + 학습이 얼마나 잘되었는지 알 수 있는 것
* 머신 러닝의 최종 목적은 과거의 데이터를 토대로 새로운 데이터를 예측하는 것입니다. 
* 즉, 새로운 데이터에 사용할 모델을 만드는 것이 최종 목적이므로 
* 테스트셋을 만들어 정확한 평가를 병행하는 것이 매우 중요합니다.    

* 초음파 광물 예측 데이터를 만든 세즈노프스키 교수가 실험 결과를 발표한 논문의 일부
* 은닉층(Number of Hidden Units) 개수가 올라감에 따라 
* 학습셋의 예측률(Average Performance on Training Sets)과 
* 테스트셋의 예측률(Average Performance on Testing Sets)이 어떻게 변하는지
    + 은닉층이 늘어날수록 학습셋의 예측률이 점점 올라가다가 
    + 결국 24개의 층에 이르면 100% 예측률을 보입니
* 딥러닝, 머신 러닝의 목표는 학습셋에서만 잘 작동하는 모델을 만드는 것이 아닙니다. 
* 새로운 데이터에 대해 높은 정확도를 안정되게 보여 주는 모델을 만드는 것이 목표
* 데이터를 이용해 성능을 향상시키려면 우선 충분한 데이터를 가져와 추가하면 됩니다. 
* 데이터를 추가하는 것 자체가 어렵거나 데이터 추가만으로는 성능에 한계가 있을 수 있
    + 따라서 가지고 있는 데이터를 적절히 보완해 주는 방법을 사용
    + 교차 검증 방법을 사용해 가지고 있는 데이터를 충분히 이용하는 방법

## 학습셋과 테스트셋 분리

In [8]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [9]:
from sklearn.model_selection import train_test_split

# 학습셋과 테스트셋을 구분합니다.
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, shuffle=True)

In [10]:
# 모델을 다시 설정합니다.
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [11]:
# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', 
metrics=['accuracy'])

In [12]:
# 모델을 실행합니다.
history = model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=False)

In [13]:
score = model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

2/2 [==============================] - 0s 5ms/step - loss: 0.5855 - accuracy: 0.8571
Test accuracy: 0.8571428656578064


## 모델 저장과 재사용
* 학습이 끝난 후 지금 만든 모델을 저장하면 언제든 이를 불러와 다시 사용할 수
* hdf5 파일 포맷은 주로 과학 기술 데이터 작업에서 사용되는데, 
  크고 복잡한 데이터를 저장하는 데 사용

In [14]:
# pip install h5py

In [15]:
# pip install cython

In [16]:
# 학습 결과를 저장하려면 model.save() 함수를 이용해 모델 이름을 적어 저장
# 모델 이름과 저장할 위치를 함께 지정합니다.
model.save('data/model/model10.hdf5')

In [17]:
from tensorflow.keras.models import load_model

In [18]:
# 테스트를 위해 조금 전 만든 모델을 메모리에서 삭제
del model

In [19]:
# load_model() 함수를 사용해서 조금 전 저장한 모델을 불러옵니다.
# 모델이 저장된 위치와 이름까지 적어 줍니다.
model = load_model('./data/model/model10.hdf5')

In [20]:
# 불러온 모델을 테스트셋에 적용해 정확도를 구합니다.
score = model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

2/2 [==============================] - 0s 5ms/step - loss: 0.5855 - accuracy: 0.8571
Test accuracy: 0.8571428656578064


## k겹 교차 검증
* 데이터가 충분히 많아야 모델 성능도 향상
* 실제 프로젝트에서는 데이터를 확보하는 것이 쉽지 않거나 많은 비용이 발생하는 경우도 있습
* 가지고 있는 데이터를 십분 활용하는 것이 중요
    + 특히 학습셋을 70%, 테스트셋을 30%로 설정할 경우 
    + 30%의 테스트셋은 학습에 이용할 수 없다는 단점
* k겹 교차 검증이란 데이터셋을 여러 개로 나누어 하나씩 테스트셋으로 사용하고 
* 나머지를 모두 합해서 학습셋으로 사용하는 방법
* 가지고 있는 데이터의 100%를 학습셋으로 사용할 수 있고, 
  또 동시에 테스트셋으로도 사용할 수 있습

In [21]:
# 데이터를 원하는 수만큼 나누어 각각 학습셋과 테스트셋으로 사용되게 하는 함수는 
# 사이킷런 라이브러리의 KFold() 함수
from sklearn.model_selection import KFold

# 몇 겹으로 나눌 것인지 정합니다.
k = 5 

# KFold 함수를 불러옵니다. 분할하기 전에 샘플이 치우치지 않도록 섞어 줍니다.
kfold = KFold(n_splits=k, shuffle=True) 

# 정확도가 채워질 빈 리스트를 준비합니다.
acc_score = []

In [22]:
def modelOne():
    model = Sequential() # 딥러닝 모델의 구조를 시작합니다.
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [23]:
# k겹 교차 검증을 이용해 k번의 학습을 실행합니다.
# for 문에 의해 k번 반복합니다.
# split()에 의해 k개의 학습셋, 테스트셋으로 분리됩니다.
for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]  
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = modelOne()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=0) 
    
    accuracy = model.evaluate(X_test, y_test)[1] # 정확도를 구합니다.
    acc_score.append(accuracy)                   # 정확도 리스트에 저장합니다.

2/2 [==============================] - 0s 5ms/step - loss: 0.4862 - accuracy: 0.8537


In [24]:
# k번 실시된 정확도의 평균을 구합니다.
avg_acc_score = sum(acc_score) / k

# 결과를 출력합니다.
print('정확도: ', acc_score)
print('정확도 평균: ', avg_acc_score)

정확도:  [0.9047619104385376, 0.8095238208770752, 0.8809523582458496, 0.6829268336296082, 0.8536585569381714]
정확도 평균:  0.8263646960258484
